# ML-7 Оптимизация гиперпараметров модели
###  Содержание <a class="anchor" id=0></a>
- [1. Введение](#1)
- [2. Базовая оптимизация](#2)
- [2.1 GridSearch](#2-1)
- [2.2 RandomSearch](#2-2)
- [2.3 Рекомендации по настройкам](#2-3)
- [3. Продвинутая оптимизация](#3)
- [3.1 HUPEROPT](#3-1)
- [3.2 OPTUNA](#3-2)
- [4. Практика](#4)
- [5. Итоги](#5)


# 1. Введение <a class="anchor" id=1></a>

[к содержанию](#0)

Итак, как мы уже неоднократно упоминали ранее, в машинном обучении есть два типа параметров.

**Внутренние** (параметры модели)

Подбираются во время обучения и определяют, как использовать входные данные для получения необходимого результата.

Например, это веса (коэффициенты уравнения) в линейной/логистической регрессии.

**Внешние** (параметры алгоритма)

Их принято называть `гиперпараметрами`. Внешние параметры могут быть произвольно установлены перед началом обучения и контролируют внутреннюю работу обучающего алгоритма.

Например, это параметр регуляризации в линейной/логистической регрессии.

`Гиперпараметры` отвечают за сложность взаимосвязи между входными признаками и целевой переменной, поэтому сильно влияют на модель и качество прогнозирования.

Продемонстрируем это на примере задачи регрессии с помощью двух графиков работы алгоритма случайного леса, построенного на основе `5`, `100` деревьев (`n_estimators = [5, 100]`):

<img src=ml7_img1.png>

Видим, что при 100 деревьях модель находит более сложную закономерность в данных и точность соответственно будет выше, чем при 5.

Каждый алгоритм МО имеет набор гиперпараметров, которые определяют, как именно он строит модель на обучающей выборке. Например, в модуле ML-2 для повышения эффективности модели мы уже рассматривали подбор параметра регуляризации `alpha` для алгоритма линейной регрессии `Ridge`.

In [1]:
import numpy as np
import matplotlib as plt
from sklearn import linear_model
from sklearn import metrics

In [2]:
#Создаем список из 20 возможных значений от 0.001 до 10
alpha_list = np.linspace(0.01, 10, 20)
#Создаем пустые списки, в которые будем добавлять результаты 
train_scores = []
test_scores = []
for alpha in alpha_list:
    #Создаем объект класса линейная регрессия с L2-регуляризацией
    ridge_lr_poly = linear_model.Ridge(alpha=alpha, max_iter=10000)
    #Обучаем модель предсказывать логарифм целевого признака
    ridge_lr_poly.fit(X_train_scaled_poly, y_train_log)
    #Делаем предсказание для каждой из выборок
    #Если обучили на логарифме, то от результата необходимо взять обратную функцию - экспоненту
    y_train_predict_poly = np.exp(ridge_lr_poly.predict(X_train_scaled_poly))
    y_test_predict_poly = np.exp(ridge_lr_poly.predict(X_test_scaled_poly))
    #Рассчитываем метрику для двух выборок и добавляем их в списки
    train_scores.append(metrics.mean_absolute_error(y_train, y_train_predict_poly))
    test_scores.append(metrics.mean_absolute_error(y_test, y_test_predict_poly))
 
#Визуализируем изменение R^2 в зависимости от alpha
fig, ax = plt.subplots(figsize=(12, 4)) #фигура + координатная плоскость
ax.plot(alpha_list, train_scores, label='Train') #линейный график для тренировочной выборки
ax.plot(alpha_list, test_scores, label='Test') #линейный график для тестовой выборки
ax.set_xlabel('Alpha') #название оси абсцисс
ax.set_ylabel('MAE') #название оси ординат
ax.set_xticks(alpha_list) #метки по оси абцисс
ax.xaxis.set_tick_params(rotation=45) #поворот меток на оси абсцисс
ax.legend(); #отображение легенды

NameError: name 'X_train_scaled_poly' is not defined

<img src=ml7_img2.png>

Наилучшее значение метрики соответствует `alpha = 0.01` (кстати, можно попробовать перебрать значения `alpha < 0.01`).

В данном случае мы просто воспользовались циклом for и перебрали некоторые заданные значения alpha, хотя, по всей видимости, не самые оптимальные. Поэтому подобранные эмпирическим путём значения гиперпараметров с большей вероятностью дадут низкую прогностическую эффективность.

Также рассмотренный метод визуализации зависимости метрики от гиперпараметра позволяет выбрать только один внешний параметр, в данном случае — `alpha`. А что делать, если у нас не один, а несколько? 

Например, вспомним основные внешние параметры `DecisionTreeClassifier`:

* `criterion` — критерий информативности. Может быть равен `'gini'` — критерий Джини — и `'entropy'` — энтропия Шеннона.
* `max_depth` — максимальная глубина дерева. По умолчанию `None`, глубина дерева не ограничена.
* `max_features` — максимальное число признаков, по которым ищется лучшее разбиение в дереве. По умолчанию `None`, то есть обучение производится на всех признаках.
* `min_samples_leaf` — минимальное число объектов в листе. По умолчанию — `1`.

Мы, конечно, можем сделать кучу вложенных циклов. Однако, поскольку поиск оптимальных значений гиперпараметров является общераспространенной задачей МО, библиотека `scikit`-`learn` и другие предлагают методы, позволяющие её решить.

Тщательный подбор гиперпараметров гарантирует, что модель покажет максимально возможную точность на обучающих данных, **но это совершенно не означает хороший результат на тестовых или новых данных.**

Поиск оптимальных значений гиперпараметров модели является сложной задачей, обязательной почти для всех моделей и наборов данных. Однако важно понимать смысл гиперпараметров перед их подбором.

# 2. Базовая оптимизация <a class="anchor" id=2></a>

[к содержанию](#0)

В базовой оптимизации, предоставляемой библиотекой `sklearn`, есть два основных метода — `grid search` и `random search`. С ними мы сейчас и познакомимся. Оба используются при решении реальных задач, поэтому важно разобраться, как они устроены. 

Наиболее часто используемый метод — это **поиск по сетке** (`grid search`), который по сути является попыткой перебрать все возможные комбинации заданных гиперпараметров. Мы указываем список значений для различных гиперпараметров, и, ориентируясь на нашу метрику, оцениваем эффективность модели для каждого их сочетания, чтобы получить оптимальную комбинацию значений.

Допустим, мы хотим подобрать гиперпараметры `min_samples_leaf` и `max_depth` для алгоритма `DecisionTreeClassifier`. Зададим списки их значений:

In [ ]:
min_samples_leaf = [3, 5, 8, 9]
max_depth = [4, 5, 6, 7, 8]

Поскольку нам нужно перебрать четыре различных значения для `min_samples_leaf` и пять — для `max_depth`, то получается всего `4*5=20` комбинаций. Модель будет обучена 20 раз; столько же раз будет рассчитана метрика.

Сетка выглядит следующим образом:

<img src=ml7_img3.png>

# ОПАСНОСТЬ ПЕРЕОБУЧЕНИЯ И УТЕЧКИ ДАННЫХ

Для того, чтобы выбрать оптимальные значения гиперпараметров, мы ориентируемся на выбранную метрику, рассчитанную на тестовой выборке. Мы делали это для подбора гиперпараметра регуляризации `alpha`, но является ли это надёжным подходом?

>Эту проблему мы уже обсуждали в модуле ML-5 «Валидация и оценка качества моделей».

Давайте вспомним: мы перебираем множество значений гиперпараметров и выбираем ту комбинацию значений, которая даёт наилучшую точность на тестовых данных. **Однако это совсем не означает, что на новых данных мы получим такой же результат**. 

Поскольку мы использовали тестовый набор для настройки гиперпараметров, мы больше не можем использовать его для оценки качества модели. Теперь в этих целях нам необходим независимый набор данных, то есть набор, который не использовался для построения модели и настройки её гиперпараметров.

Следовательно, надо разбить данные на **три части**: **обучающую** для построения модели, проверочную (**валидационную**) для выбора гиперпараметров модели, а также **тестовую** для оценки качества модели и выбранных гиперпараметров. 

<img src=ml7_img4.png>

Наличие всех трёх наборов данных критически важно для использования МО. Любой подбор гиперпараметров, сделанный на тестовых данных, «сливает» модели информацию, содержащуюся в них, и может привести к неправильной оценке качества модели. Такая проблема относится к категории утечки данных, которую мы уже тоже затрагивали в модуле по валидации.

>Рассмотренный метод разбиения данных на обучающий, проверочный и тестовый наборы является вполне рабочим и относительно широко используемым, но весьма чувствителен к равномерности разбиения данных. 

Для лучшей оценки обобщающей способности вместо одного разбиения данных на обучающий и проверочный наборы мы можем воспользоваться перекрёстной проверкой, то есть **кросс-валидацией** (`cross` `validation`). В таком случае качество модели оценивается для каждой комбинации гиперпараметров по всем разбиениям кросс-валидации. 

<img src=ml7_img5.png>

>Пояснение к рисунку. Предположим, что у нас есть `n` комбинаций гиперпараметров. Берём первую комбинацию и обучаем на них первую модель с помощью кросс-валидации с 10 фолдами (`cv=10`), затем рассчитываем метрику как среднее по всем разбиениям. Так проделываем для каждой комбинации и выбираем ту, при которой наша метрика наилучшая. В итоге мы обучим `n*cv` моделей, но выберем один набор гиперпараметров, который и будет использоваться для обучения итоговой модели на всей обучающей выборке.

# GRIDSEARCHCV

Поскольку поиск по сетке с кросс-валидацией является весьма распространённым методом настройки гиперпараметров, библиотека `scikit`-`learn` предлагает класс `GridSearchCV`, в котором осуществляется именно такой вариант.

## Файл `ML-7.Optimization_of_hyperparameters.ipynb`

Основные параметры `GridSearchCV`:

* `estimator` — алгоритм, который будем оптимизировать;
* `param_grid` — словарь или список словарей. Словарь с именами гиперпараметров (в формате строки (`str`), например, '`max_depth`') в качестве ключей и списками параметров (например, `[5, 8, 10]`) в качестве значений. Итого: `{'max_depth': [5, 8, 10] }`.

Также можно передать список таких словарей:

In [ ]:
param_grid = [
              {'max_depth': [5, 8, 10],
               'min_samples_leaf': [7, 8, 9] } #первый словарь 
              {'n_estimators': [100, 200, 300], 
               'max_depth': [5, 8, 10] } #второй словарь 
             ]

В таком случае каждый словарь в списке перебирается отдельно и последовательно. Это позволяет выполнять поиск по любой последовательности настроек параметров.

* `scoring` — по умолчанию используется score-функция заданного алгоритма:

* * для классификации — `sklearn.metrics.accuracy_score`;

* * для регрессии — `sklearn.metrics.r2_score`;

>Возможно выбрать любую другую в зависимости от условий задачи. Различные варианты смотрите [здесь.](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

* `cv` — количество фолдов в кросс-валидации, по умолчанию используется 5.

* `n_jobs` — количество ядер для распараллеливания расчёта. -1 использует все существующие ядра.

## Чтобы воспользоваться классом `GridSearchCV`, необходимо:

1. Импортировать библиотеку:

In [ ]:
from sklearn.model_selection import GridSearchCV

2. Указать искомые гиперпараметры в виде словаря  `param_grid`: ключами словаря являются имена настраиваемых гиперпараметров, а значениями – тестируемые настройки гиперпараметров. Мы рассмотрим сетку из:

* '`penalty`' — тип регуляризации. Может принимать значения `l1`,  `l2`, '`elasticnet`' или `None` (отсутствие регуляризации);
* '`solver`' — алгоритм оптимизации, может принимать значения '`newton`-`cg`', '`lbfgs`', 
* '`liblinear`', '`sag`', '`saga`', по умолчанию — '`lbfgs`'.

Важно помнить, что выбор алгоритма оптимизации зависит от выбранного типа штрафа:

<img src=ml7_img6.png>

In [ ]:
param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
                  'solver': ['lbfgs', 'saga'] #алгоритм оптимизации
                  }

3. Вызвать класс `GridSearchCV` и передать модель (`LogisticRegression`), сетку искомых параметров (`param_grid`), а также число фолдов, которые мы хотим использовать в кросс-валидации, и `n_jobs = -1`, чтобы использовать все доступные ядра для расчётов:

In [ ]:
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=1, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)

In [ ]:
grid_search.fit(X_train_scaled, y_train) 
#Затраченное время: 1min 4s

Созданный нами объект `grid_search` аналогичен классификатору, поэтому мы можем вызвать стандартные методы `fit`, `predict` и `score` от его имени. Однако, когда мы вызываем `fit()`, он запускает кросс-валидацию для каждой комбинации гиперпараметров, указанных в `param_grid`:

>`GridSearchCV` включает в себя не только поиск лучших параметров, но и автоматическое построение новой модели на всём обучающем наборе данных с использованием параметров, которые дают наилучшее значение метрики при кросс-валидации.

Наилучшая найденная комбинация гиперпараметров сохраняется в атрибуте `best_params_`:

In [ ]:
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
# Наилучшие значения гиперпараметров: {'penalty': 'none', 'solver': 'lbfgs'}

In [ ]:
# Наилучшая метрика:
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test_scaled, y_test)))

Либо можем посмотреть любую другую метрику, воспользовавшись методом `predict()` и передав предсказанные значения в функцию для расчёта метрики (например, `f1_score()`): 

In [ ]:
y_test_pred = grid_search.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
 
# accuracy на тестовом наборе: 0.84
# f1_score на тестовом наборе: 0.64

Значения метрик не изменились, но это значит лишь, что мы не нашли комбинацию внешних параметров лучше, чем заданы по умолчанию. Это неудивительно, и достаточно часто исходные  гиперпараметры дают неплохой результат, но это не повод останавливаться!

Попробуем расширить сетку гиперпараметров и проделаем те же шаги:

# `RANDOMIZEDSEARCHCV` <a class="anchor" id=4></a>

[к содержанию](#0)

Альтернативным подходом подбора различных комбинаций гиперпараметров в библиотеке `scikit`-`learn` является `RandomizedSearchCV`. 

Рандомизированный поиск работает почти так же, как решётчатый поиск, но он гораздо экономичнее и эффективнее по времени, потому что мы можем задать количество подбираемых комбинаций, а не брать все возможные.

<img src=ml7_img7.png>

На этой картинке изображено принципиальное различие двух методов: 

* В `GridSearchCV` сетка задаётся вручную, перебираются различные значения гиперпараметров с каким-то шагом, в итоге получается что-то похожее на «красивую» сетку слева на картинке. Однако минимум функции (белое пятно) мы так и не обнаруживаем — а ведь он где-то рядом, возможно, просто между подобранными нами комбинациями.

* `RandomizedSearchCV` выбирает `n` (количество задаем сами) случайных точек/комбинаций из заданных нами последовательностей. Как следствие, мы можем перебирать не все возможные точки, а только часть из них, тем самым управляя скоростью работы перебора.

Основные параметры `RandomizedSearchCV` аналогичны `GridSearchCV`, за исключением `n_iter`:

* `estimator` — алгоритм, который будем оптимизировать;

* `param_grid` — словарь или список словарей.

>Примечание. В ранних версиях `sklearn` данный параметр был обозначен как `param_grid`.

* `scoring` — по умолчанию используется score-функция заданного алгоритма:

* * для классификации — `sklearn.metrics.accuracy_score`;

* * для регрессии — `sklearn.metrics.r2_score`;

* `cv` — количество фолдов в кросс-валидации, по умолчанию используется 5.

* `n_jobs` — количество ядер для распараллеливания расчёта. -1 использует все существующие ядра.

* `n_iter` — количество комбинаций на расчёт. От этого параметра напрямую зависит время оптимизации и качество модели.

# РЕКОМЕНДАЦИИ ПО НАСТРОЙКЕ ГИПЕРПАРАМЕТРОВ АНСАМБЛЕЙ НАД РЕШАЮЩИМИ ДЕРЕВЬЯМИ <a class="anchor" id=2-3></a>

[к содержанию](#0)

## АЛГОРИТМ СЛУЧАЙНОГО ЛЕСА (`RANDOMFOREST`)

* `n_estimators` — число итераций (количество деревьев). Частично работает правило «чем больше, тем лучше», но иногда это не имеет особого смысла и сильно увеличивает затраты, поэтому стоит пробовать обучать сотни деревьев [100,200, 300, 400]. Если нет изменений, то оставить минимальное — 100.

* `max_depth` — максимальная глубина дерева. В случайном лесе строятся «сильные» деревья, каждое из которых даёт полноценный прогноз, поэтому глубина деревьем может быть достаточно большой. Стоит следить за переобучением.

* `max_features` — максимальное количество признаков, учитываемых алгоритмом при поиске лучшего разделения;

* `subsample` — доля выборки, которая будет использоваться для обучения каждого алгоритма — дерева.

## АЛГОРИТМ ГРАДИЕНТНОГО БУСТИНГА (`GRADIENTBOOSTING`)

* `n_estimators` — число итераций (количество деревьев) : хотя ошибка на обучении монотонно стремится к нулю, ошибка на контроле, как правило, начинает увеличиваться после определенной итерации. Оптимальное число итераций можно выбирать, например, по отложенной выборке или с помощью кросс-валидации.

* `learning_rate` — темп обучения (0;1]:

>На практике оказывается, что градиентный бустинг очень быстро строит композицию, ошибка которой на обучении выходит на асимптоту (достигает предела), после чего начинает настраиваться на шум и переобучаться. Параметр `learning_rate` контролирует, насколько сильно каждое дерево будет пытаться исправить ошибки предыдущих деревьев. Более высокая скорость обучения означает, что каждое дерево может внести более сильные корректировки. Как правило, чем меньше темп обучения, тем лучше качество итоговой композиции.

* `max_depth` — максимальная глубина дерева. Используется для борьбы с переобучением. Рекомендуется устанавливать не более 5.

* `max_features` — максимальное количество признаков, учитываемых алгоритмом при поиске лучшего разделения.

* `subsample` — доля выборки, которая будет использоваться для обучения каждого алгоритма. Это ещё один способ улучшения качества градиентного бустинга. Таким образом вносится рандомизация в процесс обучения базовых алгоритмов, что снижает уровень шума в обучении, а также повышает эффективность вычислений. 

<img src=ml7_img8.png>

Главное отличие техник `Bagging` и `Boosting` состоит в параллельном и последовательном построении деревьев соответственно.

Основные параметры градиентного бустинга деревьев — это количество деревьев (`n_estimators`) и скорость обучения (`learning_rate`), контролирующие степень вклада каждого дерева в устранение ошибок предыдущих деревьев. Эти два параметра тесно взаимосвязаны, поскольку более низкое значение `learning_rate` означает, что для построения модели аналогичной сложности необходимо большее количество деревьев.

>В отличие от случайного леса, в котором более высокое значение `n_estimators` всегда дает лучшее качество, увеличение значения `n_estimators` в градиентном бустинге даёт более сложную модель, что может привести к переобучению. При всём этом случайный лес, в отличие от градиентного бустинга, использует глубокие деревья, способные сформировать полноценный прогноз. 

Общепринятая практика для бустинга — подгонять `n_estimators` в зависимости от бюджета времени и памяти, а затем подбирать различные значения `learning_rate`.

# 3. Продвинутая оптимизация <a class="anchor" id=3></a>

[к содержанию](#0)

Один из способов  — это **байесовская оптимизация**. Она отличается от случайного поиска или поиска по сетке тем, что учитывает предыдущие результаты, а не выбирает комбинации из вариантов, не имеющих информации о прошлых оценках. Во многих случаях это позволяет найти лучшие значения гиперпараметров модели за меньшее количество времени. Таким образом, мы получаем и более быструю оптимизацию, и более качественный результат. Это два желаемых результата, особенно когда мы работаем с настройкой гиперпараметров моделей МО.

Существует несколько разных алгоритмов для этого типа оптимизации, но особенно используемым является `Tree-Structured Parzen Estimators` (`TPE`).

## TREE-STRUCTURED PARZEN ESTIMATORS (`TPE`)

1. На каждой итерации алгоритм `TPE` учитывает информацию о прошлых опробованных комбинациях гиперпараметров и только потом принимает решение, какой набор следует попробовать дальше. 

Чтобы приступить к использованию `TPE`, необходимо выполнить несколько итераций с помощью случайного поиска. 

2. На следующем шаге происходит разделение собранных наборов на две группы:

* в первую группу входят наборы, дающие наилучшие результаты после оценки;
* во вторую — все остальные.


На изображении ниже: первая группа — **красные** точки находятся в области минимума целевой функции; вторая группа — **синие** точки, все остальные.

<img src=ml7_img9.png>

>Основная цель алгоритма — найти набор гиперпараметров, который с большей вероятностью будет в первой группе и с меньшей вероятностью во второй группе. Таким образом, для принятия следующего решения используется целое распределение наилучших комбинаций — **красные точки на графике**.

3. Далее `TPE` моделирует вероятности правдоподобия для каждой из групп, используя **формулу Байеса**:

<img src=ml7_img10.png>

4. Затем, используя вероятность правдоподобия из первой группы, отбирается набор комбинаций, которые с большей вероятностью попадут в первую группу и с меньшей вероятностью — во вторую. 

<img src=ml7_img11.png>

5. **Шаги 2-4**  будет выполняться до тех пор, пока не будет достигнуто максимальное количество итераций. 

В итоге мы найдём наилучшую комбинацию гиперпараметров.

## `HYPEROPT`<a class="anchor" id=3-1></a>

[к содержанию](#0)

`Hyperopt` — это библиотека `Python` с открытым исходным кодом на основе байесовской оптимизации, в которой реализован алгоритм `Tree-Structured Parzen Estimators` (`TPE`).

Три шага для использования `Hyperopt`:

1. Задание пространства поиска гиперпараметров. 

Объявляем список гиперпараметров, тип распределения и его границы.

Основные типы:

In [ ]:
hp.choice(label, options) #равновероятный выбор из множества

hp.randint(label, upper) #случайное целое число; random seed, например 

hp.uniform(label, low, high) #равномерное непрерывное распределение

hp.normal(label, mu, sigma) #нормальное непрерывное распределение

hp.lognormal(mu, sigma) #логнормальное непрерывное распределение

Используйте для:

* **категориальных** — `hp.choice`;

* **целочисленных** —  `hp.randit`, `hp.quniform`;

* **непрерывных** — аналогично целочисленным и `hp.normal`, `hp.uniform`, `hp.lognormal`, `hp.loguniform`.

2. Задание целевой функции. 

Создаём модель МО, передаём ей данные и оцениваем её на основе выбранной метрики. Можем минимизировать/максимизировать значение метрики.

3. Задание алгоритма поиска:

* `Random Search`.

* `Tree of Parzen Estimators `(`TPE`).

Полезные ссылки: 

* [Hyperopt: A Python Library for Optimizing the Hyperparameters of Machine Learning Algorithms](https://lms.skillfactory.ru/assets/courseware/v1/32710b2c3069452833036ea0e4da10fe/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/bergstra_hyperopt.pdf)

* [Hyperopt на GitHub](https://github.com/hyperopt/hyperopt/)

* [Байесовский ниндзя (Хабр)](https://habr.com/ru/post/494242/)

Устанавливаем библиотеку
!pip install hyperopt
или
!conda install -c conda-forge hyperopt

In [1]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))


Версия Hyperopt : 0.2.7


Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [2]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

Интерфейс `Hyperopt` отличается от `Grid` или `RandomizedSearch`, поэтому нам нужно создать функцию для минимизации. Она должна принимать словарь значений гиперпараметров и возвращать значение целевой функции.

In [3]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train_scaled, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

NameError: name 'X_train_scaled' is not defined

In [ ]:
# начинаем подбор гиперпараметров
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

#100%|██████████| 20/20 [01:10<00:00,  3.50s/it, best loss: -0.7986892215038526]
#Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}
#CPU times: user 1min 10s, sys: 183 ms, total: 1min 10s
#Wall time: 1min 10s

In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test_scaled, y_test)))
y_test_pred = model.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

#f1_score на обучающем наборе: 0.80
#accuracy на тестовом наборе: 0.86
#f1_score на тестовом наборе: 0.68

Мы можем продолжить поиск гиперпараметров, чтобы получить лучшие результаты.

Для байесовских оптимизаторов это возможно, так как они ориентируются на прошлые результаты: предыдущие входные данные для целевой функции и результирующие потери.

`Hyperopt` продолжит поиск с того места, где он остановился, если мы передадим ему объект `Trials`, который уже содержит информацию о предыдущих запусках.

**Всегда сохраняйте свои предыдущие результаты!**

In [ ]:
# если необходимо продолжить подбор, 
# то увеличиваем max_evals(должен быть строго больше, чем на предыдущих итерациях) 
# и используем старый trials
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=23, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

#100%|██████████| 3/3 [00:08<00:00,  2.89s/it, best loss: -0.7986892215038526]
#Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}

Видим, что `Hyperopt` смог улучшить нашу метрику, причём за меньшее время.

In [ ]:
# отрисуем, как менялась точность при различных гиперпараметрах
tpe_results=np.array([[x['result']['loss'],
                      x['misc']['vals']['max_depth'][0],
                      x['misc']['vals']['n_estimators'][0]] for x in trials.trials])

tpe_results_df=pd.DataFrame(tpe_results,
                           columns=['score', 'max_depth', 'n_estimators'])
# тепловая карта в данном случае не очень наглядна, возьмем линейный график
tpe_results_df.plot(subplots=True,figsize=(10, 10));

#array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f2883875490>,
       #<matplotlib.axes._subplots.AxesSubplot object at 0x7f28838d9d90>,
       #<matplotlib.axes._subplots.AxesSubplot object at 0x7f288412bf90>],

<img src=ml7_img12.jpeg>

По графикам видно, что лучшая точность достигается именно в `best_params`. Однако заметьте, что метрика отрицательная, так как нам необходимо максимизировать нашу метрику, в то время как `Hyperopt` может только минимизировать.

## OPTUNA <a class="anchor" id=3-2></a>

[к содержанию](#0)

[Сайт Optuna](https://optuna.org/)

`Optuna` — это достаточно новый фреймворк/библиотека, разработанный специально для оптимизации гиперпараметров. Помимо байесовских алгоритмов, есть возможность удаления плохих комбинаций из рассмотрения. По умолчанию алгоритм удаляет комбинации, в которых модель даёт качество ниже медианы из уже рассмотренных. Optuna помогает  быстрее находить лучшие гиперпараметры и работает с большинством современных известных библиотек `ML`, таких как `scikit`-`learn`, `xgboost`, `PyTorch`, `TensorFlow`, `skorch`, `lightgbm`, `Keras`, `fast-ai `и другими.

## Три шага для использования `Optuna`: 

**1. Задание пространства поиска гиперпараметров.**

>Основные функции:
>
>* `suggest_categorical(name, choices)` — для категориальных гиперпараметров;
>* `suggest_int(name,low,high,step=1,log=False)` — для целочисленных гиперпараметров;
>* `suggest_float(name,low,high,step=None,log=False)` — для непрерывных гиперпараметров;
>* `suggest_uniform(name,low,high)` — для целочисленных и непрерывных гиперпараметров.
>
>С помощью необязательных аргументов `step` и `log` можно дискретизировать или взять логарифм целочисленных и непрерывных параметров.

**2. Задание целевой функции.**

Создаём модель МО, передаём ей данные и оцениваем её на основе выбранной метрики, можем минимизировать/максимизировать значение метрики. На данном этапе будет обучена модель только на одной комбинации гиперпараметров.

**3. Создание объекта исследования create study.** 

По умолчанию используется алгоритм поиска `TPE` (есть и другие варианты) и вызов метода `optimize()`, в который передаётся целевая функция, созданная на первом шаге. Выполняется заданное `n_trials` раз, подставляются различные комбинации гиперпараметров.

<img src=ml7_img12.png>

# 4. Практика <a class="anchor" id=4></a>

[к содержанию](#0)

Наша практика будет основана на соревновании [Kaggle: Predicting a Biological Response](https://www.kaggle.com/c/bioresponse)

Данные представлены в формате `CSV`.  Каждая строка представляет молекулу. 

* Первый столбец `Activity` содержит экспериментальные данные, описывающие фактический биологический ответ `[0, 1]`; 

* Остальные столбцы `D1-D1776` представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

**Предварительная обработка не требуется**, данные уже закодированы и нормализованы.

В качестве метрики будем использовать `F1-score`.

Необходимо обучить две модели: `логистическую регрессию` и `случайный лес`. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (`GridSeachCV`, `RandomizedSearchCV`, `Hyperopt`, `Optuna`) хотя бы по разу, максимальное количество итераций не должно превышать `50`.

# 5. Итоги <a class="anchor" id=5></a>

[к содержанию](#0)